---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

# Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. 

**Objective**: to predict whether a given blight ticket will be paid on time.

All data has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/).

___

The two data files for use in training and validating your models are train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from scipy.stats import randint

def blight_model():
    
    #############################################################################################################################
    
    # Define cleaning_train_data function for the preprocessing of the training data
    
    def cleaning_train_data(data):
        
        # Eliminate tickets from outside the USA
    
        data = data[train_data.country == 'USA'].copy()
    
        # Eliminate tickets when compliance is null
        
        data = data[np.isnan(data.compliance) == 0].copy()
    
        # Drop some of the features
    
        data = data.drop(['agency_name', 'inspector_name', 'violator_name', 'violation_street_name','violation_street_number', 'violation_zip_code', 'mailing_address_str_name', 'mailing_address_str_number', 'city', 'state', 'zip_code', 
                                 'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date', 'violation_code', 'violation_description', 'disposition', 'admin_fee', 'state_fee', 
                                 'late_fee', 'clean_up_cost', 'grafitti_status', 'payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 'compliance_detail'], axis=1).copy()
    
        # Apply dictionaries to ticket_id to get latitude and longitude
                
        addresses_df=pd.read_csv("readonly/addresses.csv", skiprows=1,header=None)
        #addresses_df=pd.read_csv("addresses.csv", skiprows=1,header=None)
        addresses_dict=dict(zip(list(addresses_df.iloc[:,0]), list(addresses_df.iloc[:,1])))
        
        latlons_df=pd.read_csv("readonly/latlons.csv", skiprows=1,header=None)
        #latlons_df=pd.read_csv("latlons.csv", skiprows=1,header=None)
        latitude_dict=dict(zip(list(latlons_df.iloc[:,0]), list(latlons_df.iloc[:,1])))
        longitude_dict=dict(zip(list(latlons_df.iloc[:,0]), list(latlons_df.iloc[:,2])))

        data['addresses']= data['ticket_id'].map(addresses_dict)
        data['latitude']= data['addresses'].map(latitude_dict)
        data['longitude']= data['addresses'].map(longitude_dict)
        data[['latitude']]=data[['latitude']].replace(np.nan, np.mean(data[['latitude']]))
        data[['longitude']]=data[['longitude']].replace(np.nan, np.mean(data[['longitude']]))
        
        scaler = MinMaxScaler()
        data[['latitude','longitude']] = scaler.fit_transform(data[['latitude','longitude']])
        data = data.drop(['ticket_id','addresses'], axis=1).copy()        
      
        # Eliminate rows with any NaN value
        
        data = data.dropna().copy()
        
        # Apply MinMax scaler
        
        #scaler = MinMaxScaler()
        #data[['fine_amount','discount_amount','judgment_amount']] = scaler.fit_transform(data[['fine_amount','discount_amount','judgment_amount']])
        
        return data
    
    #############################################################################################################################
    
    # Define cleaning_test_data function for the preprocessing of the test data
    
    def cleaning_test_data(data):
        
        # Drop name variables
    
        data = data.drop(['agency_name', 'inspector_name', 'violator_name', 'violation_street_name','violation_street_number', 'violation_zip_code', 'mailing_address_str_name', 'mailing_address_str_number', 'city', 'state', 'zip_code', 
                                 'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date', 'violation_code', 'violation_description', 'disposition', 'admin_fee', 'state_fee', 
                                 'late_fee', 'clean_up_cost', 'grafitti_status'], axis=1).copy()
    
        # Apply dictionaries to ticket_id to get latitude and longitude
                
        addresses_df=pd.read_csv("readonly/addresses.csv", skiprows=1,header=None)
        #addresses_df=pd.read_csv("addresses.csv", skiprows=1,header=None)
        addresses_dict=dict(zip(list(addresses_df.iloc[:,0]), list(addresses_df.iloc[:,1])))
        
        latlons_df=pd.read_csv("readonly/latlons.csv", skiprows=1,header=None)
        #latlons_df=pd.read_csv("latlons.csv", skiprows=1,header=None)
        latitude_dict=dict(zip(list(latlons_df.iloc[:,0]), list(latlons_df.iloc[:,1])))
        longitude_dict=dict(zip(list(latlons_df.iloc[:,0]), list(latlons_df.iloc[:,2])))

        data['addresses']= data['ticket_id'].map(addresses_dict)
        data['latitude']= data['addresses'].map(latitude_dict)
        data['longitude']= data['addresses'].map(longitude_dict)
        data[['latitude']]=data[['latitude']].replace(np.nan, np.mean(data[['latitude']]))
        data[['longitude']]=data[['longitude']].replace(np.nan, np.mean(data[['longitude']]))
        
        scaler = MinMaxScaler()
        data[['latitude','longitude']] = scaler.fit_transform(data[['latitude','longitude']])
        data = data.drop(['ticket_id','addresses'], axis=1).copy()
        
        # Apply MinMax scaler
        
        #scaler = MinMaxScaler()
        #data[['fine_amount','discount_amount','judgment_amount']] = scaler.fit_transform(data[['fine_amount','discount_amount','judgment_amount']])
        
        return data
    
    #############################################################################################################################
    
    
    # Read training data
    
    train_data = pd.read_csv("train.csv", encoding = 'ISO-8859-1',
                             dtype={'zip_code': pd.np.str,'non_us_str_code': pd.np.str,'grafitti_status': pd.np.str})
    
    # Clean training data
    
    clean_train_data = cleaning_train_data(train_data)
    
    ##########################################
    
    # Select option for machine learning model
    
    ##########################################
    
    # 1) Random forest 
    
    clf = RandomForestClassifier().fit(clean_train_data.drop(['compliance'], axis=1), clean_train_data['compliance'])
    
    # 1.2) Random forest optimized
    
    #clf_initial = RandomForestClassifier()
    
    # specify parameters and distributions to sample from
    #param_dist = {"max_depth": [3, None],
              #"bootstrap": [True, False],
              #"criterion": ["gini", "entropy"]}
    
    #clf_random_search = RandomizedSearchCV(clf_initial, param_distributions=param_dist,
                                   n_iter=4, cv=5, iid=False)
    
    #clf = clf_random_search.fit(clean_train_data.drop(['compliance'], axis=1), clean_train_data['compliance'])
    
    ##########################################
    
    # 2) Gradient boosted decision trees
    
    #clf = GradientBoostingClassifier().fit(clean_train_data.drop(['compliance'], axis=1), clean_train_data['compliance'])
    
    ##########################################
    
    # 3) Linear regression optimized with regularization and c parameter
    
    #clf_initial = LogisticRegression()
    #grid_values = {'penalty': ['l1', 'l2'], 'C':[0.1, 1, 10]}
    #clf = GridSearchCV(clf_initial, param_grid = grid_values, scoring='roc_auc')
    #clf.fit(clean_train_data.drop(['compliance'], axis=1), clean_train_data['compliance'])    
    
    ##########################################    
    
    # Get the score
    
    clf.score(clean_train_data.drop(['compliance'], axis=1), clean_train_data['compliance'])
    
    # Read test data
    
    #test_data = pd.read_csv("test.csv")
    test_data = pd.read_csv("readonly/test.csv")
    
    # Clean test data
    
    clean_test_data = cleaning_test_data(test_data)  
    
    # Return predicted probabilities
    
    #return pd.Series(clf.predict_proba(clean_test_data)[:,1],index=test_data['ticket_id']).rename('compliance').replace(np.nan, 0.1)

    # Return score in training set
    
    return clf.score(clean_train_data.drop(['compliance'], axis=1), clean_train_data['compliance'])

blight_model()

0.93413982698334264

In [ ]:
blight_model()